# AutoDiff Documentation


## Sections:

[I. Introduction](#Introduction)

[II. Background](#Background)

[III. Software Organization](#Software_Organization)

[IV. How to Use](#How_to_Use)

[V. Implementation](#Implementation)

[VI. Testing](#Testing)

[VII. Implemented Extensions](#Implemented_Extensions)

[VIII. Future Features](#Future_Features)

[IX. References](#References)

<a id='Introduction'></a>
## I. Introduction

The purpose of our software and of automatic differentiation is to efficiently evaluate derivatives at machine precision.
There are two main alternatives to automatic differentiation ("AD") for the evaluation of derivatives: symbolic differentiation and numerical (approximation) methods.
While symbolic differentiation evaluates derivatives to machine precision, it is often costly to evaluate and prone to 'expression swell': 
symbolic differentiation may result in exponentially large expressions to evaluate. In contrast, numerical methods (such as the finite difference method) are 
quick to evaluate but do not generally result in machine precision: when evaluating $(f(x+\epsilon)-f(x))/\epsilon$, use of a large epsilon results in imprecise approximation,
whereas use of a small epsilon results in floating point errors. Moreover, the complexity of numerical methods increases as the number of dimensions of the gradient increases.<sup>3</sup>
Automatic differentiation circumvents these challenges by "accumulating" numerical evaluations of simple elementary functions.
    
The efficient evaluation of derivatives at machine precision naturally has a wide (and growing) range of application areas, of which a few are highlighted below:
* Differentiation of functions implemented as code: AD does not require closed-form expressions (as is the case in symbolic differentiation)<sup>3</sup>
* Backpropagation: training neural networks requires finding weights that minimize the objective function
    * AD has seen increasingly widespread use since ~2015 as deep learning grows in popularity (e.g. AD implementations in TensorFlow and PyTorch)<sup>3</sup>
    * Applications include computer vision, NLP
* Solving ODEs
* Gradient-based optimization (finding minima through step-wise updates)<sup>3</sup>
* Root finding, including Newton's method

<a id='Background'></a>
## II. Background

Suppose we have a function $f: \mathbb{R}^n \to \mathbb{R}^m$

$$
f(\mathbf{x})=
\begin{bmatrix}
    f_1(x_1, ..., x_n) \\
    f_2(x_1, ..., x_n) \\
    \vdots \\
    f_m(x_1, ..., x_n)
\end{bmatrix}
$$

and suppose that we seek ${\frac{\partial f}{\partial x_1}}|_{\mathbf{x}=\mathbf{a}}$.

If we have $J_f$, the Jacobian matrix of $f$, we can compute ${\frac{\partial f}{\partial x_1}}|_{\mathbf{x}=\mathbf{a}}$ as the product of the Jacobian matrix and the desired seed vector $s$:

$$
J_f\mathbf{s}= \begin{bmatrix}
    \frac{\partial f_1}{\partial x_1} & \frac{\partial f_1}{\partial x_2} & \dots & \frac{\partial f_1}{\partial x_n} \\[1ex]
    \frac{\partial f_2}{\partial x_1} & \frac{\partial f_2}{\partial x_2} & \dots & \frac{\partial f_2}{\partial x_n} \\[1ex]
    \vdots & \vdots & \ddots & \vdots \\[1ex]
    \frac{\partial f_m}{\partial x_1} & \frac{\partial f_m}{\partial x_2} & \dots & \frac{\partial f_m}{\partial x_n}
\end{bmatrix}
\begin{bmatrix}
1 \\[1ex]
0 \\[1ex]
\vdots \\[1ex]
0
\end{bmatrix}
= \begin{bmatrix}
\frac{\partial f_1}{\partial x_1} \\[1ex]
\frac{\partial f_2}{\partial x_1} \\[1ex]
\vdots \\[1ex]
\frac{\partial f_m}{\partial x_1}
\end{bmatrix}
$$

Rather than using matrix-vector multiplication or an explicit matrix representation of the Jacobian, the forward mode of AD involves calculating ${\frac{\partial f}{\partial x_1}}|_{\mathbf{x}=\mathbf{a}}$ 
(i.e.  a single column of the Jacobian) by computing ${\frac{\partial f_i}{\partial x_1}}|_{\mathbf{x}=\mathbf{a}}$ for each of $i=1,...,m$ separately in one or more *forward "passes"*
(also sometimes referred to as "sweeps").<sup>2,3</sup> If $f: \mathbb{R}^1 \to \mathbb{R}^m$, only one forward pass would be needed to calculate the full Jacobian, since in that case $n=1$, the Jacobian matrix is $m$x$1$,
and the seed vector would have a single entry. If $f: \mathbb{R}^n \to \mathbb{R}^m$, then $n$ forward passes would be required (one forward pass for each entry in the $n$x$1$ seed vector). 
Although the matrix representation of the Jacobian is not explicitly used in the forward mode, this intuition carries over: within each forward pass, the derivative of
one of the variables is set to 1 and the rest to zero (for example, $\dot{x}_1=1$ if we seek ${\frac{\partial f}{\partial x_1}}$)

AD works by breaking down the function $f$ into a composition of much simpler elemental functions (or elemental operations).
Each forward pass starts by assigning 1 to the derivative of the variable $x_i$ for which we want to differentiate $f$ with respect to, and assigning 0 to all other variables.
Then the elemental functions are evaluated from innermost to outermost, accumulating two numerical values at each intermediate variable $v_i$ in the trace<sup>3,4</sup>:
* the **"primal"**: the value of the elemental function at $\mathbf{x}=\mathbf{a}$, denoted $v_i$ (i.e. the input of the elemental function will be carried over from any inner functions evaluated earlier in the trace)
* the **derivative** or "tangent", denoted $\dot{v_i}$ (i.e. the derivative of the elemental function evaluated at $\mathbf{x}=\mathbf{a}$)

Within each step, the derivative $\dot{v_i}$ is implicitly calculated using the chain rule. For example, if the elemental function corresponding to $v_3$ results from the composition of several functions $w_3(w_2(\mathbf{x}), w_1(\mathbf{x}))$ which for clarity we denote $w_3(u_1, u_2)$ then:
$$
\dot{v_3} = {\frac{\partial}{\partial x_1}} w_3(w_2(\mathbf{x}), w_1(\mathbf{x})) = {\frac{\partial w_3}{\partial u_1}}{\frac{\partial w_2}{\partial x_1}} + {\frac{\partial w_3}{\partial u_2}}{\frac{\partial w_1}{\partial x_1}}
$$
In forward mode, the innermost elemental function primals and derivatives are evaluated first and 'accumulated' outwards.

<a id='Software_Organization'></a>
## III. Software Organization

The main project directory contains the `config.sh` file. This file has the 
`cmake` and `make install` commands that build the library, check environment, run test suite 
in `AutoDiff`. The `config.sh` file is specified in `.travis.yml`, enabling 
TravisCI to build our tests each time we `git push`. Our test suite also be utilized 
by `CodeCov` to show how much of our code we've hit, which utilizes `AD_COV.info`.

There are two main ways to package cplusplus software: `cmake` and `make`. We choose `CMake` 
because the user has to do so much less in order to link up our library. `Make` requires us to 
configure the `VPATH` and locate the `.so` file. `CMake` requires less user configuration. 

Our `AutoDiff/` folder is what the user needs to have locally in order to utilize our AutoDiff library.
To build, the user will first use "./config" in the root directory. It will then generate two folders: 
`build` and `install` under AutoDiff. 
The user will need to `cd` into the `build/` folder, run the `cmake` command, then edit the `Makefile`, 
adding the `target_link_libraries` command to link up our library. Then, all the user needs to do is 
add `#include <AutoDiff.h>` at the top of their `.cpp` file, and `C++` will know where to find our 
AutoDiff library.

Below is a diagram of our directory stucture. 

```
cs107-FinalProject/
            | README.md
            | config.sh
            | docs/
                | README.md
                | milestone1.ipynb
                | milestone2_progress.ipynb
                | milestone2.ipynb
            | .travis.yml
            | App/
            | Dual/
                | CMakeLists.txt
                | config.sh
                | coverage.sh
                | include/
                    | Dual.h
                | core/
                    | src/
                        | Dual.cpp
                        | CMakeLists.txt
                    | tests
                        | CMakeLists.txt
                        | test_main.cpp
                        | src/
                            | CMakeLists.txt
                            | test_DUAL.cpp
                            | test_vars.h
                    | CMakeLists.txt
            | AutoDiff/
                | CMakeLists.txt
                | config.sh
                | coverage.sh
                | include/
                    | AutoDiff.h
                | core/
                    | src/
                        | AutoDiff.cpp
                        | CMakeLists.txt
                    | tests
                        | CMakeLists.txt
                        | test_main.cpp
                        | src/
                            | CMakeLists.txt
                            | test_AD.cpp
                            | test_vars.h
                    | CMakeLists.txt
```

We use GoogleTest to generate local code coverage reports. We use `converge.sh` file (in Dual/ and AutoDiff/) to make it 
eaiser to call lcov and link coverage info to an `index.html` under a generated "coverage" folder. To call
`converge.sh`, the user may run "./config.sh -gtest" in the root directory, or go to Dual/ folder then do "./coverage.sh"

<a id='How_to_Use'></a>
## IV. How to Use

### Configuration:

Under the root directory, there is a `config.sh` file which is essential to get started this application.

Default option: `./config.sh`

Cleaning: `./config.sh -c`

Run UnitTest: `./config.sh -gtest`

For help: `./config.sh -h`, and you will see a list of options to configure.

### Package Requirements:

1. cmake

tutorial: https://cliutils.gitlab.io/modern-cmake/chapters/intro/installing.html

documentation: https://cmake.org/cmake/help/v3.19/

useful links: https://github.com/ackirby88/CS107/tree/master/cmake

2. GoogleTest (for testing only, not required)

Under the root directory, type "cd 3PL" in the coomand prompt to go to the 3rd package library. Type "./build_3PL.sh" to install googleTest.

tutorial: https://github.com/google/googletest/tree/master/googletest

3. lcov (for testing only, not required)

Under the root directory, type "cd 3PL/lcov" in the coomand prompt to go to the lcov package. Then, type "make install" or "sudo make install"
in the command prompt to install it. When installation is completed, you may try "make test" to see if it is installed correctly.

tutorial: http://ltp.sourceforge.net/coverage/lcov.php

source code repo: https://github.com/linux-test-project/lcov

### Usage:

To use our `AutoDiff library`:

1. the user places `#include<AutoDiff.h>` at the top of their `.cpp` file wherein they intend to compute forward mode.

2. A user build up an object of `double` and `AutoDiff types`. When declaring an `AutoDiff type` object, they must specify the object value, and may optionally seed that value as the second parameter. If only one parameter value is provided, the derivative is seeded with `1.0`.

3. The user then calls `.val` (or `.getVal()`) on their object to obtain the function value and call `der` (or `.getDer()`) on their object to obtain the derivative computed by forward mode.


#### Notice: If you get a "permission denied", you need to check if config.sh, coverage.sh, or Makefile is executable. Type "chmod +x \[executable file\]" in the command prompt to add files to be executable.

### Operating system: 

1. MacOS

Current "Dual/CMakeLists.txt" and "AutoDiff/CMakeLists.txt" comment out two pieces that make it different from
Linux version. 

The 1st one is "line 10: set(CMAKE_INSTALL_RPATH ${CMAKE_INSTALL_PREFIX}/lib)". If you receive an error like "realpath: command not found", you should consider line10 out.

The 2nd one is "line 27: if(CMAKE_C_COMPILER_ID MATCHES "GNU")" and correspondingly "line 30: endif()". It maybe due to that MacOS use LLVM gcov to emulate gcov, if MacOS users don't commit these two lines out, the unittesting won't be triggered. Therefore, a code coverage report website cannot be generated.

2. Linux

Uncomment the two parts (line10, line27, line30) and it should work. 

### Usage Examples (Setup)

``git clone https://github.com/Crimson-Coders-3/cs107-FinalProject/tree/m2b-dev``

``cd 3PL``

``./build_3PL.sh``

``cd ..``

``./config.sh ``

if you want to get test coverage, 

``./config.sh -gtest``


### Usage Examples
Please note that currently nested functions are not supported and each step in the trace has to be initialized separately. For example:

For instance this won’t work:
```
AutoDiff test = AutoDiff(4.0);
AutoDiff test1 = sin(pow(test, 3.0));
```

But this works:
```
AutoDiff test = AutoDiff(4.0);
AutoDiff test1 = pow(test, 3.0):
AutoDiff test2 = sin(test1);
```

To get the function evaluation, call `test2.val` or `test2.getVal()`. To get the function derivative by forward mode, call `test2.der` or `test2.getDer()`.

Print out the evaluation and derivative using `test2.print()`.


### External Dependencies

This project uses the Prof Kirby's skeleton project and `Math.h`.

<a id='Implementation'></a>
## V. Implementation

Our `AutoDiff.h` header file contains class declaration and function signature of elemental operations.

Our `AutoDiff class` is realized in `AutoDiff.cpp`, whereby our constructor contains two attributes: double `val` and double `der`: `val` is the value of the `AutoDiff` "variable" and `der` is initialized as its seed.

We overload the following mathematical operators within the `AutoDiff class`:
```
AutoDiff AutoDiff::operator + ( AutoDiff &obj )
AutoDiff AutoDiff::operator - ( AutoDiff &obj )
AutoDiff AutoDiff::operator * ( AutoDiff &obj )
AutoDiff AutoDiff::operator / ( AutoDiff &obj )
```

And define the following the following functions which return objects of `AutoDiff type`.
```
* AutoDiff operator + (double lhs, AutoDiff &rhs);
* AutoDiff operator - (double lhs, AutoDiff &rhs);
* AutoDiff operator * (double lhs, AutoDiff &rhs);
* AutoDiff operator / (double lhs, AutoDiff &rhs);
* AutoDiff pow ( AutoDiff &a, AutoDiff &b );
* AutoDiff pow ( AutoDiff &obj, double a );
* AutoDiff pow (double lhs, AutoDiff &rhs);
* AutoDiff exp ( AutoDiff &a );
* AutoDiff sin(AutoDiff &input);
* AutoDiff cos(AutoDiff &input);
* AutoDiff tan(AutoDiff &input);
* AutoDiff asin(AutoDiff &input);
* AutoDiff acos(AutoDiff &input);
* AutoDiff atan(AutoDiff &input);
* AutoDiff log(AutoDiff &input);
* AutoDiff sinh(AutoDiff &input);
* AutoDiff cosh(AutoDiff &input);
* AutoDiff tanh(AutoDiff &input);
```

Since each of the functions above accepts AutoDiff objects and returns AutoDiff objects containing 
an 'updated' current value and derivative, the forward mode is realized by stringing together multiple
function calls representing the composition of elemental functions which define the user's function of interest.
The output of any given elemental function is effectively the trace.

For ease of viewing the AutoDiff object, a `print` function is also defined which prints the value and derivative and can be called for example via:
```
AutoDiff test = AutoDiff(2.0);
AutoDiff output = pow(test, 3.0);
output.print();
```

#### Currently, only the single variable case is implemented (i.e. scalar functions of a single input). So, the paragraphs below are our plan, not implemented yet.

Next, we intend to expand our `AutoDiff class` to allow the vector inputs, not just scalars. We will have a second constructor wherein the user must provide a vector (of `duble type`) when declaring their `AutoDiff` object. The second parameter is a seed vector which, if left empty, will seed the vectors as unit vectors (i.e. vectors where the variable in question is `1` and every other variable is `0`).

To implement evaluation and derivative on a vector input, we intent to expand the overloaded operators and mathematical functions to accept and be able to process vector inputs of `AutoDiff type` (and double). Data structurally, a vector is an array.

After we have completed expanding from scalar to vector inputs, we need to be able to calculate evaluation and forward mode on matrices, namely calculating the Jacabian, not just a scalar function.

To calculate the Jacobian, we will expect the user create a matrix of `AutoDiff type`, requiring us to define such a data structure using a multi-dimesional array. We will create another constructor, overload math operations, and redine math functions. Our constructor will initialize the inputs as multi-dimensional arrays instead of vectors or scalars.
 
Our implementation is just being scaled to multiple dimensions, where we work with array data types instead of `double` scalar data types.

Once our AD is implemented within `AutoDiff.cpp` we essentially just need to reference the functionality to write our root finder. The Newton-Raphson method is pretty short, and involves iterating once we acquire the function evaluation and derivative evaluation. The file `roots.cpp` will handle this iteration.

Lastly, we will wrap the diff implementation along with the root finder in a file called `AutoDiff_main.cpp`.

<a id='Testing'></a>
## VI. Testing

* Location：Dual/code/tests and AutoDiff/code/tests

    * We have pre-defined several unit tests under the above directories
    * To run these tests, please (1) run the AutoDiff/code/config.sh file and (2) run the AutoDiff/code/coverage.sh file

* Tools:

    * CodeCov: 
    
        CodeCov provides statistics of code coverage so that each commit and pull request is measured and in control. 
        To do this, we need to create a .yaml file and specify our metrics (target at least 90%). 
        As our software covers two parts of functions: the basic function(AD implementation) and extensions, 
        it is reasonable to use tags in the .yaml file to separate coverage reports for different modules<sup>7</sup>. 
        Setting Codecov to send notifications to the Slack channel is also a good idea<sup>8</sup>, so everyone in the group 
        gets notified when there is an update.
    
    * TravisCI: 
        
        TravisCI is a continuous integration platform that automatically builds and tests code changes. 
        Instead of merging large pieces of code and integrate them all in the end, we want to take a continuous 
        integration approach - merge in small code changes and get immediate feedback when each commit and pull 
        request happens<sup>9</sup>. We also need a .yaml file to use it. It can also send notifications through Slack. 
* Plan: 
    * Step 1: Exploratory Testing

        Exploratory testing is a simultaneous process of test design and test execution all done at the same time, 
        contrarily to scripted testing, where all test cases are determined in advance<sup>10</sup>.
        We will note down ideas about what to test and how it can be tested in a document file (.md). 
        We will update this document with additional test case designs, critically evaluate current test results with 
        respect to our expectations, and learn from the testing constantly.

    * Step 2: Unit tests

        Unit tests isolate issues that may arise and each unit is tested independently. 
        We will define the smallest unit to a each Class method. 
        By writing tests for the smallest testable units, then the compound behaviors between those, 
        we can build up comprehensive tests for more complex applications.<sup>11</sup>

    * Step 3: Acceptance tests

        After unit testings, we want to validate software against functional requirements/specifications. We mainly concentrate on:

        * Functionality

        * Basic Usability: Can users freely navigate through the screens without any difficulties?

        * Error Conditions: Do suitable error messages display? Are they clear and proper?

        * Accessibility


        

* Evaluations:

    * Code coverage

    * Risk analysis: What are the potential risks of current implementation? What are the important ones and what need to be addressed? 

    * Test execution log: recordings on the test execution

<a id='Implemented_Extensions'></a>
## VII. Implemented Extensions

### Dual Numbers

Dual number is an interesting type of number that uses the symbol $\epsilon$ (epsilon) and offers how to do automatic
differentiation in another persepective. It automatically calculates the derivative and the value of a function at the same time! 

Dual numbers are pretty similar to imaginary numbers. A dual number has a real part and a dual part. We write

$$
x = a + b\epsilon
$$

But! For imaginary numbers, $i^2 = -1$, whereas for dual numbers, $\epsilon^2 = 0$ (and $\epsilon$ is not 0!). 

#### Basic Math

Add and subtraction of dual numbers are the same as adding complex numbers: we just add the real and dual parts separately.

$$
(3+6\epsilon) + (1+2\epsilon) = (3+1)+(6\epsilon+2\epsilon) = 4+8\epsilon
$$

$$
(3+6\epsilon) - (1+2\epsilon) = (3-1)+(6\epsilon-2\epsilon) = 2+4\epsilon
$$

To multiply dual numbers, we use F.O.I.L. just like we do with complex numbers

$$
(3+6\epsilon) \times (1+2\epsilon) = 3 + 6\epsilon +6\epsilon + 12\epsilon^2 = 3 + 12\epsilon + 12\epsilon^2
$$

As $\epsilon^2=0$, the last item $12\epsilon^2$ disappears. 

$$
(3+6\epsilon) \times (1+2\epsilon) = 3 + 12\epsilon
$$

The division process relates to the conjugate, like complex division - multiplying the denominator  by its conjugate 
in order to cancel the non-real parts.<sup>12</sup> The formula is given below:

$$
\begin{aligned}
\frac{a+b\epsilon}{c+d\epsilon} &= \frac{(a+b\epsilon)(c-d\epsilon)}{(c+d\epsilon)(c-d\epsilon)}\\
&= \frac{ac-ad\epsilon+bc\epsilon-bd\epsilon^2}{c^2 - d^2\epsilon^2}\\
&= \frac{ac-ad\epsilon+bc\epsilon-bd\epsilon^2}{c^2}\\
&= \frac{ac+(bc-ad)\epsilon}{c^2}\\
&= \frac{a}{c}+\frac{(bc-ad)\epsilon}{c^2}
\end{aligned}
$$

which is defined when $c$ is non-zero. If, on the other hand, $c$ is zero while $d$ is not, then the equation

$$
\begin{aligned}
\frac{a+b\epsilon}{c+d\epsilon} &= k\\
\frac{a+b\epsilon}{d\epsilon} &= k\\
a+b\epsilon &= kd\epsilon\\
\end{aligned}
$$

1. has no solution if $a$ is nonzero
2. otherwise is solved by any dual number of the form $k=\frac{b}{d}+y\epsilon$. To expalins this, suppose we have $k=x+y\epsilon$: 

$$
\begin{aligned}
b\epsilon &= kd\epsilon\\
b\epsilon &= (x+\epsilon y)d\epsilon\\
b\epsilon &= xd\epsilon + \epsilon^2 y\\
b\epsilon &= xd\epsilon\\
x &= \frac{b}{d}\epsilon
\end{aligned}
$$

So, $k=\frac{b}{d}+y\epsilon$ and is the value of $\frac{a+b\epsilon}{c+d\epsilon}$.

#### Using $\epsilon^2$ for AD

Suppose we have a function: $
f(x) = 3x + 2$, and calculates $f(4)$ and $f'(4)$. 

We know $4 = 4 + 1\epsilon$, using $1$ for the dual component, since $x$ has a derivative of $1$.<sup>13</sup> We take this into the function:

$$
f(4) = f(4 + \epsilon) = (4 + 1\epsilon) \times 3 + 2 + 1\epsilon = (12 + 3\epsilon) + 2 = 14 + 3\epsilon
$$

We can find out the real number component $(14)$ is the value of $f(4)$ and the dual component $(3)$ is the derivative $f'(4)$, 
which is verified below:

$$
f(x) = 3x+2 = 3\times4 + 2 = 14
$$

$$
f'(x) = 3
$$

It is not an coincidence! Now try a quadratic example: $f(x)=5x^2 + 3x + 1$, also evaluates at $f(4)$.

$$
\begin{aligned}
f(4 + \epsilon) &=5(4 + \epsilon)^2 + 3\times(4 + \epsilon) + 1 \\
 &= 5(16 + 8\epsilon+\epsilon^2 ) + 12 + 3\epsilon + 1\\
 &= 80 + 40\epsilon + 5\epsilon^2 + 12 + 3\epsilon + 1\\
 &= (80+12+1) + (40+3)\epsilon + 5\epsilon^2 \\
 &= 93 + 43\epsilon + 5\epsilon^2\\
 &= 93 + 43\epsilon
\end{aligned}
$$

Now verify it:

$$
\begin{aligned}
f(4)&=5\times 4^2 + 3\times 4 +1\\ &= 5\times 16 + 12 + 1\\ &= 80 + 12 + 1 \\&= 93
\end{aligned}
$$

$$
\begin{aligned}
f'(x)&=10x + 3\\ 
f'(4)&=10\times 4 + 3 = 43
\end{aligned}
$$

We can also use dual number on trigonometric functions to do differentiation. 
What's more, it is not only limited to the 1st derivative.
It is possible to modify this technique to get the 2nd the 3rd, or the Nth derivative of a function. But in a 2nd derivative
scenario, we need to define $\epsilon^3=0$, instead of $\epsilon^2=0$, so on and so forth.
We can also extend this to multivariable functions. To do this, we need to define $\epsilon_x$, $\epsilon_y$, ... for each variables.

To conclude, using dual numbers gives us exact derivatives, within the limitations of floating point math 
(compared to finite difference method). 

#### Dual Number Implementation

It is tempting to create a "DualNumber" Class and overloads different operators$(+,-,*,/)$ and math functions, eg. sqrt, pow, sin, cos, tan, atan, ... 
We keep all independent variables in a array-like container and in each overloading function, we loop through this array to calcualte derivative for each variables.

Just like what stated above, dual number can be expressed as the followings:

$$
z = a+b\epsilon
$$
$$
{\epsilon}^{2} = 0
$$

For the purpose of auto differentiation, we can use dual numbers in this way:

$$
z = x + x^{'}\epsilon
$$

Next, we can get a dual number cheat sheet:

| Expr. | Expr. Deri. | Expr. w Dual Num. | Expr. w Dual Num. Result | Real Part | Dual Part | Notes |
|:-----|:--------------|:-----|:-----------------------|:-------|:-------|
|  $u+v$| $u^{'}+v^{'}$  |   $(u+u^{'}\epsilon)+(v+v^{'}\epsilon)$ | $u+u^{'}\epsilon+v+v^{'}\epsilon$  | $u+v$   | $(u^{'}+v^{'})\epsilon$ |
|  $u\times v$     |    $(u^{'}v+vu^{'})$  |   $(u+u^{'}\epsilon)\times(v+v^{'}\epsilon)$    | $uv + u^{'}v\epsilon+vu^{'}\epsilon+{\epsilon}^{2}$   | $u\times v$  | $(u^{'}v+vu^{'})\epsilon$ |
|  $u-v$| $u^{'}-v^{'}$  |   $(u+u^{'}\epsilon)-(v+v^{'}\epsilon)$ |   $u+u^{'}\epsilon-v+v^{'}\epsilon$| $u-v$  | $(u^{'}-v^{'})\epsilon$ |
|  $u\div v$     |    $\frac{(u^{'}v+vu^{'})}{v^2}$ |   $(u+u^{'}\epsilon)\div(v+v^{'}\epsilon)$  |  $\frac{uv+(u^{'}v-uv^{'})\epsilon}{v^2}$   |   $u\div v$      |   $\frac{(u^{'}v-uv^{'})\epsilon}{v^2}$      |
| $\sin(u)$ | $\cos(u)u^{'}$ | $\sin(u+u^{'}e)$  | $\sin(u) +\cos(u)u^{'}\epsilon$ | $\sin(u)$ | $\cos(u)u^{'}\epsilon$ | using $\sin(u'\epsilon)=u'\epsilon,\cos(u^{'}\epsilon)=1 $
| $\cos(u)$ | $-\sin(u)u^{'}$ | $\cos(u+u^{'}e)$| $\cos(u)-\sin(u)\epsilon$ |  $\cos(u)$ | $-\sin(u)\epsilon$ |
| $\exp(u)$ | $\exp(u)$ | $\exp(u+u^{'})$ | $\exp(u)+u^{'}\exp(u)$ | $\exp(u)$ | $u^{'}\exp(u)$ | using Taylor series
| $\log(u)$ | $\log(u+u^{'})$ | $\log(u+u^{'}\epsilon)$ | $\log(u)+\frac{u^{'}}{u}\epsilon$ | $\log(u)$ | $\frac{u^{'}}{u}\epsilon$ |
| $u^{k}$   | ${ku^{'}u^{k-1}}$ |${(u+u^{'}u^{'}\epsilon)}^{k}$  | $u^{k}+{ku^{k-1}}\epsilon$|$u^{k}$ | ${ku^{'}u^{k-1}}$|

So, we can plan to make a Dual class and use a type name T to let it can be used as float, double ... etc.

<a id='Future Features'></a>
## VIII. Future Features 

### Root Finding and Optimization

#### Newton's method (also known as the Newton-Raphson Method):

##### General properties and overview
* "second order method" in that the quadratic approximation for $f$ at a local point $a$ is used to compute the descent direction
(direction in which to step to reach stationary points)
* well-suited for optimizing or finding roots of convex functions with a moderate number of inputs (e.g. linear regression, logistic regression)
* requires fewer steps to converge than methods which use linear (rather than quadratic) approximations of the function
* with non-convex functions, the algorithm may converge to local maxima
* as with other local optimization methods, starts from a single starting point and iteratively "refines" it by stepping in a descent direction
* for optimization, the idea is to start at point $\mathbf{a_i}$ and go to $\mathbf{a_{i+1}}$ by creating second order Taylor series approx. to $f$ at $a_i$ and traveling to the stationary point of this second order Tayor series approx.<sup>14</sup>

##### Theory
Building from simple to more general cases of the functional form of $f$ provides intuition for Newton's method:<sup>15, 16, 17</sup>

*$f: \mathbb{R} \to \mathbb{R}$*
$$
f(x) = f(x)
$$
* the second order Taylor approx. of $f$ at $\mathbf{a}$ is: 
$$g(x) = f(a) + \frac{d}{dx} f(a) (x-a)+\frac{1}{2}\frac{d^2}{dx^2}f(a)(x-a)^2$$
* we can solve for the stationary point of the second order Taylor approx. of $f$ by setting its derivative to 0:
$$
\begin{aligned}
\frac{d}{dx}g = \frac{d}{dx}f(a) + \frac{1}{2}\frac{d^2}{dx^2}f(a)(2x^{*} - 2a) &= 0\\
\frac{d^2}{dx^2}f(a)(x^{*} - a) &= -\frac{d}{dx}f(a)\\
x^* &= a - \frac{\frac{d}{dx}f(a)}{\frac{d^2}{dx^2}f(a)}\\
\end{aligned}
$$

*$f: \mathbb{R}^n \to \mathbb{R}$*
$$
f(\mathbf{x})= f(x_1, ..., x_n)
$$
* the second order Taylor approx of $f$ at $\mathbf{a}$ is: 
$$g(\mathbf{x}) = f(\mathbf{a}) + \nabla f(\mathbf{a})^{T}(\mathbf{x}-\mathbf{a})+\frac{1}{2}(\mathbf{x}-\mathbf{a})^{T}\nabla^2f(\mathbf{a})(\mathbf{x}-\mathbf{a})$$
* analogously to the above, solving for the stationary point of the second order Taylor approx. of $f$ yields:  
&nbsp;&nbsp;&nbsp;&nbsp;$$\mathbf{x}^* = \mathbf{a} - (\nabla^2 f(\mathbf{a}))^{-1} \nabla f(\mathbf{a})$$  
where $\nabla^2 f(\mathbf{a})$ is the Hessian of $f$ at $\mathbf{a}$

*$f: \mathbb{R}^n \to \mathbb{R}^m$*
$$
f(\mathbf{x})=
\begin{bmatrix}
    f_1(x_1, ..., x_n) \\
    f_2(x_1, ..., x_n) \\
    \vdots \\
    f_m(x_1, ..., x_n)
\end{bmatrix}
$$
* the second order Taylor approx of the $i^{\text{th}}$ element of $f$ at $\mathbf{a}$ is: 
$$g_i(\mathbf{x}) = f_i(\mathbf{a}) + \nabla f_i(\mathbf{a})^{T}(\mathbf{x}-\mathbf{a})+\frac{1}{2}(\mathbf{x}-\mathbf{a})^{T}\nabla^2f_i(\mathbf{a})(\mathbf{x}-\mathbf{a})$$

*Optimization*:<sup>14</sup>

Based on the above, we can see that to go from a starting point of $\mathbf{a}$ to the stationary point $\mathbf{x^*}$ of the quadratic approximation of $f$ at $\mathbf{a}$, we must move in the descent direction $(\nabla^2 f(\mathbf{a}))^{-1} \nabla f(\mathbf{a})$ (for a general $f: \mathbb{R}^n \to \mathbb{R}$).
This is how we can iteratively update our estimate of the minimum of $f$:

$$
\mathbf{a_{n+1}} = \mathbf{a_n} - (\nabla^2 f(\mathbf{a}))^{-1} \nabla f(\mathbf{a})
$$

*Root-finding*:

If we instead seek to find the roots of $f$, rather than setting the derivative of the second order Taylor approx. of $f$ to $0$, we can instead set the linear approx. (first order Taylor approx.) of $f$ to $\mathbf{0}$:
$$
\begin{aligned}
\mathbf{0} &= \mathbf{f(x)} + \mathbf{J(x)}(\mathbf{x^*}-\mathbf{a})\\
\mathbf{x^*} &= \mathbf{a} - \mathbf{J(x)}^{-1}\mathbf{f(x)}
\end{aligned}
$$
If $f: \mathbb{R}^n \to \mathbb{R}^m$, then the Jacobian matrix $J$ is ($m$ x $n$); if $J$ is not invertible then the pseudo-inverse $J^* = (J^TJ)^{-1}J^T$ may be used instead for iteration.
The update steps are then:<sup>15</sup>
$$
\mathbf{a_{n+1}} = \mathbf{a_n} - \mathbf{J^* f(a_n)}
$$
In the univariate case, Newton's iteration is then:
$$
a_{n+1} = a_{n} - \frac{f(a)}{f'(a)}
$$

##### (Draft) Implementation for optimization
Inputs: $f(\mathbf{x})$, max steps $m$, initial value $\mathbf{a}$
Outputs: $[a_0, a_1, a_2...a_k]$, $[f(a_0), f(a_1), ... f(a_k)]$

Pseudocode:<sup>14</sup>
* get value and gradient (functions) of $f$ from AD implementation
* get Hessian (function) of $f$
* $\text{for } i \text { in } [1,2, ...m]:$
    * evaluate value and gradient of $f$ at $a$
    * evaluate Hessian at $a$
    * $a = a - (\nabla^2 f(a))^{-1}\nabla f(a)$

### Halley’s Method (extension of Newton’s method)

Whereas Newton's method provides quadratic convergence (the number of correct digits 'doubles' every iteration), Halley's method provides cubic convergence
(the number of correct digits 'triples' every iteration). Halley's iteration is defined as:<sup>19</sup>

$$
x_{k+1} = x_k - \frac{2f(x_k)f'(x_k)}{2(f'(x_k))^2 - f(x_k)f''(x_k)}
$$

Once methods to take the second derivative are implemented, the implementation of Halley's method should be relatively straightforward given that it involves
the evaluation of deriatives at particular values (which can be initialized in AutoDiff objects).

### Solving Linear Algebraic Equations

Current popular methods of solving the problems below mostly take an iterative approach, 
e.g. Gauss-Jordan Elimination, Gaussian Elimination with Backsubstitution, LU Decomposition. Solving these problems also take large computation time; we are curious if automatic differentiation will give a better solution.

* Solution of the matrix equation $Ax=b$ for an unknown vector $x$
* Solution of more than one matrix equation $Ax_j=b_j$, for a set of vectors $x_j$, $j=0,1,..,$ each corresponding to 
a different, known right-hand side vector bj
* Calculation of the matrix $A^{-1}$ that is the matrix inverse of a square matrix $A$, i.e., $A^{-1}\cdot A=A \cdot A^{-1} =I$, 
where $I$ is the identity matrix   
* Singular value decomposition of a matrix $A$
* Linear least-squares problem

    The reduced set of equations to be solved can be written as the $N\times N$ set of equations
    
    $$
    (A^{T}\cdot A)\cdot x = (A^{T}\cdot b)
    $$

    where $A^{T}$ denotes the transpose of the matrix A.
    These quations are called the normal equations of the linear least-squares problem. 

Usually, a direct solution of the normal equations is **NOT** generally the best way to find least-squares solutions.
We would like to see how automatic differentiation performs in this task and compare the differences. 


<a id='References'></a>
## IX. References



1. http://web4.cs.ucl.ac.uk/staff/D.Barber/publications/AMLAutoDiff.pdf
2. http://www.robots.ox.ac.uk/~tvg/publications/talks/autodiff.pdf
3. "Automatic Differentiation in Machine Learning: A Survey" https://www.jmlr.org/papers/volume18/17-468/17-468.pdf
4. https://harvard-iacs.github.io/2020-CS107/lectures/lecture10/notebook/
5. https://kenndanielso.github.io/mlrefined/
6. https://people.cs.umass.edu/~domke/courses/sml/09autodiff_nnets.pdf
7. https://docs.codecov.io/docs/flags
8. https://docs.codecov.io/docs/notifications 
9. https://docs.travis-ci.com/user/for-beginners/
10. https://www.guru99.com/exploratory-testing.html
11. https://en.wikipedia.org/wiki/Unit_testing#:~:text=Unit%20tests%20are%20typically%20automated,an%20individual%20function%20or%20procedure
12. https://en.wikipedia.org/wiki/Dual_number#:~:text=Division%20of%20dual%20numbers%20is,cancel%20the%20non%2Dreal%20parts
13. https://blog.demofox.org/2017/02/20/multivariable-dual-numbers-automatic-differentiation/
14. https://kenndanielso.github.io/mlrefined/blog_posts/7_Second_order_methods/7_3_Newtons_method.html
15. http://fourier.eng.hmc.edu/e176/lectures/NM/node21.html
16. https://www.math.usm.edu/lambers/mat419/lecture9.pdf
17. http://fourier.eng.hmc.edu/e176/lectures/NM/node45.html
18. http://web.mit.edu/18.06/www/Spring17/Multidimensional-Newton.pdf
19. http://cis.poly.edu/~mleung/CS3734/s03/ch05/HalleyIteration.pdf